## 实验题目2 龙贝格(Romberg)积分法

### 参考资料

julia 数值积分 https://blog.csdn.net/m0_37816922/article/details/103475445

julia SymPy.jl https://github.com/JuliaPy/SymPy.jl

SymPy.jl example https://docs.juliahub.com/SymPy/KzewI/1.0.29/introduction/

SymPy.jl assumption https://docs.sympy.org/dev/modules/core.html#module-sympy.core.assumptions

julia calculus https://docs.juliahub.com/CalculusWithJulia/AZHbv/0.0.16/


### 代码实现

In [19]:
using Printf
using Plots
using SymPy

In [20]:
function romberg(f::Function, xlim, iternum, ϵ)
    a, b = xlim
    n = iternum

    m = 1
    h = (b - a)   # bug here
    T1 = 1 / 2 * h * (f(a) + f(b))
    @printf("\nT:%16.9f", T1)
    # missing term: f(0.5)
    # starting point error, missed i=1
    T2, C1, C2, S1, S2, R1, R2 = zeros(7)
    for i = 1:n
        ii = 2^(i - 1)
        tmpsum = 0.0
        for k = 1:ii
            tmpsum += f(a + (k - 1 / 2) * h)
        end
        T2 = 1 / 2 * T1 + 1 / 2 * h * tmpsum
        @printf("\nT:%16.9f", T2)

        S2 = 1 / 3 * (4T2 - T1)
        @printf("\tS:%16.9f", S2)

        if m > 1
            C2 = 1 / 15 * (16S2 - S1)
            @printf("\tC:%16.9f", C2)
        end

        if m > 2
            R2 = 1 / 63 * (64C2 - C1)
            @printf("\tR:%16.9f", R2)
        end
        if m > 3
            tol = abs(R2 - R1)
            if tol < ϵ
                println("\nAccuracy requirement satisfied.\n")
                return
            end
        end
        # @label PARAM_UPDATE
        R1, C1, S1, T1 = R2, C2, S2, T2
        h /= 2
        m += 1
    end
    m
end


romberg (generic function with 1 method)

### 测试代码

#### Test 1 - Simple

In [21]:
# f(x) = x^2 * exp(x)
# f(x) = 1/x
# ϵ = 1e-6
# xlim = 1, 3

# romberg(f,xlim,10,ϵ)

In [22]:
# # compute definite integration
# @syms x y z
# f = 1 / x
# @time integrate(f, (x, 1, 3)) |> float

In [23]:
# f = 1 / (1 + x^4)
# @time integrate(f, (x, 1, 3)) |> float

### 实验题目

#### 问题 1

In [24]:
iter_num = 30


f(x) = x^2 * exp(x)
ϵ = 1e-6
xlim = 0, 1
println("f(x) = x^2 * exp(x)")
romberg(f, xlim, iter_num, ϵ)

f(x) = exp(x)sin(x)
ϵ = 1e-6
xlim = 1, 3
println("f(x) = exp(x)sin(x)")
romberg(f, xlim, iter_num, ϵ)


f(x) = 4 / (1 + x^2)
ϵ = 1e-6
xlim = 0, 1
println("f(x) = 4 / (1 + x^2)")
romberg(f, xlim, iter_num, ϵ)

f(x) = 1 / (x + 1)
ϵ = 1e-6
xlim = 0, 1
println("f(x) = 1 / (x + 1)")
romberg(f, xlim, iter_num, ϵ)

f(x) = x^2 * exp(x)

T:     1.359140914
T:     0.885660616	S:     0.727833850
T:     0.760596332	S:     0.718908238	C:     0.718313197
T:     0.728890177	S:     0.718321459	C:     0.718282340	R:     0.718281850
T:     0.720935779	S:     0.718284313	C:     0.718281837	R:     0.718281829
Accuracy requirement satisfied.

f(x) = exp(x)sin(x)

T:     5.121826420
T:     9.279762907	S:    10.665741736
T:    10.520554284	S:    10.934151409	C:    10.952045388
T:    10.842043468	S:    10.949206529	C:    10.950210203	R:    10.950181074
T:    10.923093890	S:    10.950110697	C:    10.950170975	R:    10.950170352
T:    10.943398421	S:    10.950166598	C:    10.950170325	R:    10.950170315
Accuracy requirement satisfied.

f(x) = 4 / (1 + x^2)

T:     3.000000000
T:     3.100000000	S:     3.133333333
T:     3.131176471	S:     3.141568627	C:     3.142117647
T:     3.138988494	S:     3.141592502	C:     3.141594094	R:     3.141585784
T:     3.140941612	S:     3.141592651	C:     3.141592661	R:     3.141592

### 思考题

1. 略

2. 在实验 1 中二分次数和精度的关系如何？

   二分次数越多所求的精度越高，通常预设较大的二分次数来确保计算结果有足够的精度，同时也设定早停需要满足的精度要求，避免达到所需精度之后继续计算导致增加的运算量

3. 略

4. 略